<a href="https://colab.research.google.com/github/LivingstonTardzenyuy/Deep-Learning-with-TensorFlow/blob/main/07_miltestone_project_1_food_vision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Milestone Project 1: Food Vision Big.

## checkout GPU.
* Google Colab offers free GPUs, howerver, not all of them are compatiable with mixed precision training.

Google Colab offers:
* K80 (not compatible with mixed precision training)
* P100 (compatible with mixed precision training)
* T4 (compatible with mixed precision training)

In [ ]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-f3a6ffbb-d000-5696-f3e3-0d16ad8747a4)


## Get helper functions.

We've created helper functions in the past. Now we will call them and continue utilizing them..
https://github.com/mrdbourke/tensorflow-deep-learning/blob/main/extras/helper_functions.py

In [ ]:
# Download helper functions script.

!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2025-02-28 23:31:07--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2025-02-28 23:31:07 (86.7 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [ ]:
# import a series of helper functions for the notebook
from helper_functions import create_tensorboard_callback, plot_loss_curves, compare_historys

In [ ]:
# Get TensorFlow Datasets.
import tensorflow_datasets as tfds

In [ ]:
# List all avialable datasets.
datasets_list = tfds.list_builders()
print("food101" in datasets_list)

True


In [ ]:
# Load in the data (takes 5-6 minues in Google Colab).
(train_data, test_data), ds_info = tfds.load(name="food101",
                                               split=["train", "validation"],
                                               shuffle_files=True,
                                               as_supervised=True,    # data get return in tuple.
                                               with_info=True
                                             )

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

In [ ]:
## Features of Food101 from TFDS.
ds_info.features

In [ ]:
# Get the class names.
class_names = ds_info.features["label"].names
class_names[:10]

## Exploring the Food101 data from TensorFlow Datasets.

To become one with our data, we want to find:

* Class names
* The shape of our input data (image tensors)
* The datatype of our input data.
* What the labels look like (e.g are they one-hot encoded or are they label encoded)
* Do the labels match up with the class names ?.


In [ ]:
# Take one sample of the train data.
train_one_sample = train_data.take(1)    # samples are in format (image_tensor, label)
train_one_sample

In [ ]:
# Output infor about our training sample.
for image, label in train_one_sample:
  print(f"Image shape: {image.shape}")
  print(f"Image datatype: {image.dtype}")
  print(f"Target class from class_names: {class_names[label]}")
  print(f"Target class from label: {label}")

In [ ]:
# What does our image tensor from TFDS's Food101 look like ?.
image

In [ ]:
# What are the min and max values of our image tensor ?.
import tensorflow as tf
tf.reduce_min(image), tf.reduce_max(image)

## Plot an image from TensorFlow Datasets

In [ ]:
# Plot an image tensor
import matplotlib.pyplot as plt

plt.imshow(image)
plt.axis("off")
plt.title(class_names[label.numpy()])

# Create preprocessing functions for our data.

Neural networks perform best when are data is in a certain way. eg. Batched, normalized etc.

However, not all data (including data from TensorFlow Datasets) is in the way we want it.

So in order to get it ready for our neural network we'll have to write preprocessing function and map it with our data.

What we know about our data:

* In 'unit8' datatype.
* Comprised of all different size of tensors (different sized images)
* Not scaled (the pixel values are between 0-255)


What we know models like:
* Data in 'float32' dtype (or for mixed precision 'float16' and 'float32')
* For batches, TensorFlow likes all of the tensors within a batch to be of the same size.
* Scaled (values between 0-1) also called Normalized/scaling tensors generally perform better.

With thses points in mind, we've got a few things to tackle with a preprocesing function.

Since we've been using EfficientNetBx pretrained model from tf.keras.applications we don't need to rescale our data(these architectures have rescaling built-in).

This means our function have to

* reshape our iamges to same size.
* Convert the dtype of our image tensors from 'unit8' to float32.


In [ ]:
# Make a function for preprocessing our image.
def preprocess_img(image, label, img_shape = 224):
  """
    Converts image datatype from 'uint8' to 'float32' and reshapes image to [img_shape, img_shape, color_channels].
  """

  # reshape our image size.
  image = tf.image.resize(image, [img_shape, img_shape])

  # converting our images from unit8 to float32.
  image = tf.cast(image, tf.float32)
  # image = image/255. # Scale image values. But its not required here since EfficientNetB0 already have rescalling build in.
  return image, label

In [ ]:
# Preprocess a single sample image and check the outputs.
preprocessed_img = preprocess_img(image, label)
print(f"Image before preprocessing:\n {image[:2]}...,  \nShape: {image}, \nDatatype: {image.dtype}")
print(f"Image after preprocessing:\n {preprocessed_img[0][:2]}...,  \nShape: {preprocessed_img[0].shape}, \nDatatype: {preprocessed_img[0].dtype}")


Image before preprocessing:
 [[[71 60 56]
  [70 59 55]
  [70 59 55]
  ...
  [65 60 67]
  [65 60 67]
  [64 59 66]]

 [[62 51 47]
  [64 53 49]
  [66 55 51]
  ...
  [65 60 67]
  [65 60 67]
  [64 59 66]]]...,  
Shape: [[[71 60 56]
  [70 59 55]
  [70 59 55]
  ...
  [65 60 67]
  [65 60 67]
  [64 59 66]]

 [[62 51 47]
  [64 53 49]
  [66 55 51]
  ...
  [65 60 67]
  [65 60 67]
  [64 59 66]]

 [[61 50 46]
  [65 54 50]
  [68 57 53]
  ...
  [66 61 68]
  [65 60 67]
  [63 58 65]]

 ...

 [[63 64 66]
  [64 65 67]
  [67 67 69]
  ...
  [45 42 49]
  [45 42 49]
  [45 42 49]]

 [[64 64 66]
  [64 64 66]
  [65 65 67]
  ...
  [44 41 48]
  [44 41 48]
  [44 41 48]]

 [[62 62 64]
  [62 62 64]
  [62 62 64]
  ...
  [43 40 47]
  [42 39 46]
  [42 39 46]]], 
Datatype: <dtype: 'uint8'>
Image after preprocessing:
 [[[ 65.81122   54.811222  50.811222]
  [ 73.331635  62.331635  58.331635]
  [104.015305  89.80102   86.80102 ]
  ...
  [ 61.85712   59.071384  65.64286 ]
  [ 65.        60.        67.      ]
  [ 64.64282   5

## Batch & prepare datasets.

We're now going to make our data input pipeline run really fast.

For more resources on this: https://www.tensorflow.org/guide/data_performance

In [ ]:
# Map preprocessing function to training (and parallelize)
train_data = train_data.map(map_func = preprocessed_img, num_parallel_calls= tf.data.AUTOTUNE)

# Shuffle train_data and turn it into batches and prefetch.
trian_data = train_data.shuffle(
    buffer_size= 1000         # Amount of data to shuffle at a time.
    )